In [ ]:
import sys, os
import numpy as np

In [ ]:
def compute_bias(x1, x2, xi, k):
    """
    Parameters:
    x1    first bias potential position (nm)
    x2    second bias potential position (nm)
    xi    snapshots position (nm)
    k     bias potential (kJ mol^-1 nm^-2)
    """
    b0 = k*0.5*((x1 - xi)**2.)
    b1 = k*0.5*((x2 - xi)**2.)
    return b0-b1


In [ ]:
temperature = 300.
kB = 1.381e-23 * 6.022e23 / 1000.0 # Boltzmann constant in kJ/mol/K
beta = 1.0 / (kB * temperature) # inverse temperature of simulations (in 1/(kJ/mol))

umbrella = np.arange(0.0,0.8,0.1)
lambdas = np.arange(0.0,1.05,0.05)
rvalues = []
lams = []
for k in range(2):    # 2 directions: pull + push
    for i in umbrella:
        for j in range(len(lambdas)):
                rvalues.append(i)
for j in range(len(lambdas)):   # no harmonic restraints ensembles
    rvalues.append(float(-1))
for k in range(2):    # 2 directions: pull + push
    for i in range(len(umbrella)):
        for j in range(len(lambdas)):
            lams.append(j)
for j in range(len(lambdas)):    # no harmonic restraints ensembles
    lams.append(j)
sub_rvalues=[]
for i in range(17):      # only the second half lambdas ensembles 11*8*2+11
    for j in range(10,21):
        sub_rvalues.append(j+i*21)


In [ ]:
K = len(sub_rvalues)   # number of ensembles in total
stride = 1000
os.system('mkdir frames_%d/'%(int(200000./stride)))
rang = np.arange(0, 200001, stride)  # number of frames of each ensemble
N = len(rang)
u_kn = np.zeros((K,N*K), np.float64)
for k in range(K):
    if k <= 175:   # ensemble with harmonic restraints and lambdas 0th - 175th : 11*8*2
        print 'k', k
        for l in range(K):
            if l <= 175:   # ensemble with harmonic restraints and lambdas (0th-175th)
                print 'l', l
                print 'RUN', sub_rvalues[l]
                traj = np.load('RUN%d/distance.npy'%sub_rvalues[l])[::stride]
                filename='RUN%d/dhdl.xvg'%sub_rvalues[l]
                dhdl=[]
                energy=[]
                with open(filename) as f:
                    lines=f.readlines()
                filename1 = 'RUN%d/energy.xvg'%sub_rvalues[l]
                with open(filename1) as f1:
                    lines1=f1.readlines()
                for line in rang:
                    dhdl.append(lines[line].strip().split())
                    energy.append(lines1[line].strip().split())


                for n in range(N):
                    u = float(dhdl[n][2:][int(lams[sub_rvalues[k]])])

                    u_kn[k,n+N*l] = (compute_bias(rvalues[sub_rvalues[k]],rvalues[sub_rvalues[l]],traj[n],200.) + u) * beta
            else:   # ensembles with lambdas only
                print 'no umbrella', l
                print 'RUN', sub_rvalues[l]
                traj = np.load('special/RUN%d/distance.npy'%(sub_rvalues[l]-336))[::stride]
                filename='special/RUN%d/dhdl.xvg'%(sub_rvalues[l]-336)
                dhdl=[]
                with open(filename) as f:
                    lines=f.readlines()
                for line in rang:
                    dhdl.append(lines[line].strip().split())
                for n in range(N):
                    u = float(dhdl[n][2:][int(lams[sub_rvalues[k]])])
                    u_kn[k,n+N*l] = (compute_bias(rvalues[sub_rvalues[k]],traj[n],traj[n],200.) + u) * beta
    else:    # ensembles with lambdas only
        print 'k', k
        for l in range(K):
            if l <= 175:     # ensemble with harmonic restraints and lambdas (0th-175th)
                print 'l', l
                print 'RUN', sub_rvalues[l]
                traj = np.load('RUN%d/distance.npy'%sub_rvalues[l])[::stride]
                filename='RUN%d/dhdl.xvg'%sub_rvalues[l]
                dhdl=[]
                with open(filename) as f:
                    lines=f.readlines()
                for line in rang:
                    dhdl.append(lines[line].strip().split())
                for n in range(N):
                    u = float(dhdl[n][2:][int(lams[sub_rvalues[k]])])
                    u_kn[k,n+N*l] =  u * beta

            else:   # ensembles with lambdas only
                print 'no umbrella', l
                print 'RUN', sub_rvalues[l]
                traj = np.load('special/RUN%d/distance.npy'%(sub_rvalues[l]-336))[::stride]
                filename='special/RUN%d/dhdl.xvg'%(sub_rvalues[l]-336)
                dhdl=[]
                with open(filename) as f:
                    lines=f.readlines()
                for line in rang:
                    dhdl.append(lines[line].strip().split())
                for n in range(N):
                    u = float(dhdl[n][2:][int(lams[sub_rvalues[k]])])
                    u_kn[k,n+N*l] = u * beta

np.save('frames_%d/u_kn.npy'%(int(200000./stride)),u_kn)
